In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

#  Glue context is a wrapper on the spark context

args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

datasource0 = glueContext.create_dynamic_frame.from_catalog(database = "edu_db", table_name = "score", transformation_ctx = "datasource0")
datasource1 = glueContext.create_dynamic_frame.from_catalog(database = "edu_db", table_name = "subject", transformation_ctx = "datasource1")
datasource2 = glueContext.create_dynamic_frame.from_catalog(database = "edu_db", table_name = "st_id", transformation_ctx = "datasource2")

#if there is a null field, delete it.
dropnullfields0 = DropNullFields.apply(frame = datasource0, transformation_ctx = "dropnullfields0")
dropnullfields1= DropNullFields.apply(frame =datasource1, transformation_ctx = "dropnullfields1")
dropnullfields2 = DropNullFields.apply(frame = datasource2, transformation_ctx = "dropnullfields2")

datasink0 = glueContext.write_dynamic_frame.from_options(frame = dropnullfields0, connection_type = "s3", connection_options = {"path": "s3://dev/score_etl"}, format = "parquet", transformation_ctx = "datasink0")
datasink1 = glueContext.write_dynamic_frame.from_options(frame = dropnullfields1, connection_type = "s3", connection_options = {"path": "s3://dev/subject_etl"}, format = "parquet", transformation_ctx = "datasink1")
datasink2 = glueContext.write_dynamic_frame.from_options(frame = dropnullfields2, connection_type = "s3", connection_options = {"path": "s3://dev/st_id_etl"}, format = "parquet", transformation_ctx = "datasink2")

job.commit()